In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import re
from imblearn.over_sampling import SMOTE
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer

In [ ]:
df1_temp = pd.read_csv('https://raw.githubusercontent.com/ghazafm/SocialMediaSentiment/main/Datasets/labeled%20data/Anies%20Baswedan.csv',encoding= 'ISO-8859-1')
df_anis = pd.read_csv('https://raw.githubusercontent.com/ghazafm/SocialMediaSentiment/main/Datasets/original%20data/Anies%20Baswedan.csv',encoding= 'ISO-8859-1')
df2_temp = pd.read_csv('https://raw.githubusercontent.com/ghazafm/SocialMediaSentiment/main/Datasets/labeled%20data/Prabowo%20Subianto.csv',encoding= 'ISO-8859-1')
df_prabowo = pd.read_csv('https://raw.githubusercontent.com/ghazafm/SocialMediaSentiment/main/Datasets/original%20data/Prabowo%20Subianto.csv',encoding= 'ISO-8859-1')
df3_temp = pd.read_csv('https://raw.githubusercontent.com/ghazafm/SocialMediaSentiment/main/Datasets/labeled%20data/Ganjar%20Pranowo.csv',encoding= 'ISO-8859-1')
df_ganjar = pd.read_csv('https://raw.githubusercontent.com/ghazafm/SocialMediaSentiment/main/Datasets/original%20data/Ganjar%20Pranowo.csv',encoding= 'ISO-8859-1')

In [ ]:
df_anis['label'] = df1_temp['label']
df_prabowo['label'] = df2_temp['label']
df_ganjar['label'] = df3_temp['label']

df_anis = df_anis[['Tweet','label']]
df_prabowo = df_prabowo[['Tweet','label']]
df_ganjar = df_ganjar[['Tweet','label']]

In [ ]:
df_anis.head()
df_prabowo.head()
df_ganjar.head()

In [ ]:
df_anis.describe().T

In [ ]:
df_prabowo.describe().T

In [ ]:
df_ganjar.describe().T

In [ ]:
df_anis.tail(20)

In [ ]:
df_anis.duplicated().tail(20)

In [ ]:
df_anis.describe().T

In [ ]:
df_prabowo.describe().T

In [ ]:
df_ganjar.describe().T

In [ ]:
plt.figure(figsize=(9,7))
ax = sns.countplot(x = 'label', hue ="label", data=df_anis)
plt.show()

In [ ]:
plt.figure(figsize=(9,7))
ax = sns.countplot(x = 'label', hue ="label", data=df_prabowo)
plt.show()

In [ ]:
plt.figure(figsize=(9,7))
ax = sns.countplot(x = 'label', hue ="label", data=df_ganjar)
plt.show()

In [ ]:
pd.crosstab(df_anis['label'], df_anis['label'])

In [ ]:
df_anis.isna().sum()

In [ ]:
df_prabowo.isna().sum()

In [ ]:
df_ganjar.isna().sum()

In [ ]:
anis_p = df_anis[df_anis['label']=='Positive']
anis_n = df_anis[df_anis['label']=='Negative']

In [ ]:
print(anis_p.shape,anis_n.shape)

In [ ]:
anis_n.sample(2)

In [ ]:
anis_n.Tweet.sample(5)

Ensemble with undersampling

In [ ]:
df_anis.label.value_counts()

In [ ]:
anis_x = df_anis.drop('label',axis='columns')
anis_y = df_anis['label']

In [ ]:
prabowo_x = df_prabowo.drop('label',axis='columns')
prabowo_y = df_prabowo['label']

In [ ]:
ganjar_x = df_ganjar.drop('label',axis='columns')
ganjar_y = df_ganjar['label']

In [ ]:
anis_x_train, anis_x_test, anis_y_train, anis_y_test = train_test_split(anis_x, anis_y, test_size=0.2, random_state=42, stratify= anis_y)
prabowo_x_train, prabowo_x_test, prabowo_y_train, prabowo_y_test = train_test_split(prabowo_x, prabowo_y, test_size=0.2, random_state=42, stratify= prabowo_y)
ganjar_x_train, ganjar_x_test, ganjar_y_train, ganjar_y_test = train_test_split(ganjar_x, ganjar_y, test_size=0.2, random_state=42, stratify= ganjar_y)

In [ ]:
print(anis_y_train.value_counts().Positive / anis_y_train.value_counts().Negative )
print(anis_y_train.value_counts())


In [ ]:
print(prabowo_y_train.value_counts().Positive / prabowo_y_train.value_counts().Negative )
print(prabowo_y_train.value_counts())

In [ ]:
print(ganjar_y_train.value_counts().Positive / ganjar_y_train.value_counts().Negative )
print(ganjar_y_train.value_counts())

In [ ]:
df_anis_temp = anis_x_train.copy()
df_anis_temp['label'] = anis_y_train

In [ ]:
df_anis_p = df_anis_temp[df_anis_temp.label == "Positive"]
df_anis_n = df_anis_temp[df_anis_temp.label == "Negative"]

In [ ]:
df_anis_p.sample(len(df_anis_n)).shape

In [ ]:
def get_train_batch(df_majority, df_minority, start, end):
    df_train = pd.concat([df_majority[start:end], df_minority], axis=0)
    
    x_train = df_train.drop('label')
    y_train = df_train['label']